In [10]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 18: Snailfish ---</h2><p>You descend into the ocean trench and encounter some <a href="https://en.wikipedia.org/wiki/Snailfish" target="_blank">snailfish</a>. They say they saw the sleigh keys! They'll even tell you which direction the keys went if you help one of the smaller snailfish with his <em><span title="Snailfish math is snailfish math!">math</span> homework</em>.</p>
<p>Snailfish numbers aren't like regular numbers. Instead, every snailfish number is a <em>pair</em> - an ordered list of two elements. Each element of the pair can be either a regular number or another pair.</p>
<p>Pairs are written as <code>[x,y]</code>, where <code>x</code> and <code>y</code> are the elements within the pair. Here are some example snailfish numbers, one snailfish number per line:</p>
<pre><code>[1,2]
[[1,2],3]
[9,[8,7]]
[[1,9],[8,5]]
[[[[1,2],[3,4]],[[5,6],[7,8]]],9]
[[[9,[3,8]],[[0,9],6]],[[[3,7],[4,9]],3]]
[[[[1,3],[5,3]],[[1,3],[8,7]]],[[[4,9],[6,9]],[[8,2],[7,3]]]]
</code></pre>
<p>This snailfish homework is about <em>addition</em>. To add two snailfish numbers, form a pair from the left and right parameters of the addition operator. For example, <code>[1,2]</code> + <code>[[3,4],5]</code> becomes <code>[[1,2],[[3,4],5]]</code>.</p>
<p>There's only one problem: <em>snailfish numbers must always be reduced</em>, and the process of adding two snailfish numbers can result in snailfish numbers that need to be reduced.</p>
<p>To <em>reduce a snailfish number</em>, you must repeatedly do the first action in this list that applies to the snailfish number:</p>
<ul>
<li>If any pair is <em>nested inside four pairs</em>, the leftmost such pair <em>explodes</em>.</li>
<li>If any regular number is <em>10 or greater</em>, the leftmost such regular number <em>splits</em>.</li>
</ul>
<p>Once no action in the above list applies, the snailfish number is reduced.</p>
<p>During reduction, at most one action applies, after which the process returns to the top of the list of actions. For example, if <em>split</em> produces a pair that meets the <em>explode</em> criteria, that pair <em>explodes</em> before other <em>splits</em> occur.</p>
<p>To <em>explode</em> a pair, the pair's left value is added to the first regular number to the left of the exploding pair (if any), and the pair's right value is added to the first regular number to the right of the exploding pair (if any). Exploding pairs will always consist of two regular numbers. Then, the entire exploding pair is replaced with the regular number <code>0</code>.</p>
<p>Here are some examples of a single explode action:</p>
<ul>
<li><code>[[[[<em>[9,8]</em>,1],2],3],4]</code> becomes <code>[[[[<em>0</em>,<em>9</em>],2],3],4]</code> (the <code>9</code> has no regular number to its left, so it is not added to any regular number).</li>
<li><code>[7,[6,[5,[4,<em>[3,2]</em>]]]]</code> becomes <code>[7,[6,[5,[<em>7</em>,<em>0</em>]]]]</code> (the <code>2</code> has no regular number to its right, and so it is not added to any regular number).</li>
<li><code>[[6,[5,[4,<em>[3,2]</em>]]],1]</code> becomes <code>[[6,[5,[<em>7</em>,<em>0</em>]]],<em>3</em>]</code>.</li>
<li><code>[[3,[2,[1,<em>[7,3]</em>]]],[6,[5,[4,[3,2]]]]]</code> becomes <code>[[3,[2,[<em>8</em>,<em>0</em>]]],[<em>9</em>,[5,[4,[3,2]]]]]</code> (the pair <code>[3,2]</code> is unaffected because the pair <code>[7,3]</code> is further to the left; <code>[3,2]</code> would explode on the next action).</li>
<li><code>[[3,[2,[8,0]]],[9,[5,[4,<em>[3,2]</em>]]]]</code> becomes <code>[[3,[2,[8,0]]],[9,[5,[<em>7</em>,<em>0</em>]]]]</code>.</li>
</ul>
<p>To <em>split</em> a regular number, replace it with a pair; the left element of the pair should be the regular number divided by two and rounded <em>down</em>, while the right element of the pair should be the regular number divided by two and rounded <em>up</em>. For example, <code>10</code> becomes <code>[5,5]</code>, <code>11</code> becomes <code>[5,6]</code>, <code>12</code> becomes <code>[6,6]</code>, and so on.</p>
<p>Here is the process of finding the reduced result of <code>[[[[4,3],4],4],[7,[[8,4],9]]]</code> + <code>[1,1]</code>:</p>
<pre><code>after addition: [[[[<em>[4,3]</em>,4],4],[7,[[8,4],9]]],[1,1]]
after explode:  [[[[0,7],4],[7,[<em>[8,4]</em>,9]]],[1,1]]
after explode:  [[[[0,7],4],[<em>15</em>,[0,13]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,<em>13</em>]]],[1,1]]
after split:    [[[[0,7],4],[[7,8],[0,<em>[6,7]</em>]]],[1,1]]
after explode:  [[[[0,7],4],[[7,8],[6,0]]],[8,1]]
</code></pre>
<p>Once no reduce actions apply, the snailfish number that remains is the actual result of the addition operation: <code>[[[[0,7],4],[[7,8],[6,0]]],[8,1]]</code>.</p>
<p>The homework assignment involves adding up a <em>list of snailfish numbers</em> (your puzzle input). The snailfish numbers are each listed on a separate line. Add the first snailfish number and the second, then add that result and the third, then add that result and the fourth, and so on until all numbers in the list have been used once.</p>
<p>For example, the final sum of this list is <code>[[[[1,1],[2,2]],[3,3]],[4,4]]</code>:</p>
<pre><code>[1,1]
[2,2]
[3,3]
[4,4]
</code></pre>
<p>The final sum of this list is <code>[[[[3,0],[5,3]],[4,4]],[5,5]]</code>:</p>
<pre><code>[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
</code></pre>
<p>The final sum of this list is <code>[[[[5,0],[7,4]],[5,5]],[6,6]]</code>:</p>
<pre><code>[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]
</code></pre>
<p>Here's a slightly larger example:</p>
<pre><code>[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]
</code></pre>
<p>The final sum <code>[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]</code> is found after adding up the above snailfish numbers:</p>
<pre><code>  [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
+ [7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
= [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

[[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]

- [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
  = [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]

  [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]

- [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
  = [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]]

  [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]]

- [7,[5,[[3,8],[1,4]]]]
  = [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]]

  [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]]

- [[2,[2,2]],[8,[8,1]]]
  = [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]

  [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]

- [2,9]
  = [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]]

  [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]]

- [1,[[[9,3],9],[[9,0],[0,7]]]]
  = [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]]

  [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]]

- [[[5,[7,4]],7],1]
  = [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]]

  [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]]

- [[[[4,2],2],6],[8,7]]
= [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]
</code></pre>
<p>To check whether it's the right answer, the snailfish teacher only checks the <em>magnitude</em> of the final sum. The magnitude of a pair is 3 times the magnitude of its left element plus 2 times the magnitude of its right element. The magnitude of a regular number is just that number.</p>
<p>For example, the magnitude of <code>[9,1]</code> is <code>3*9 + 2*1 = <em>29</em></code>; the magnitude of <code>[1,9]</code> is <code>3*1 + 2*9 = <em>21</em></code>. Magnitude calculations are recursive: the magnitude of <code>[[9,1],[1,9]]</code> is <code>3*29 + 2*21 = <em>129</em></code>.</p>
<p>Here are a few more magnitude examples:</p>
<ul>
<li><code>[[1,2],[[3,4],5]]</code> becomes <code><em>143</em></code>.</li>
<li><code>[[[[0,7],4],[[7,8],[6,0]]],[8,1]]</code> becomes <code><em>1384</em></code>.</li>
<li><code>[[[[1,1],[2,2]],[3,3]],[4,4]]</code> becomes <code><em>445</em></code>.</li>
<li><code>[[[[3,0],[5,3]],[4,4]],[5,5]]</code> becomes <code><em>791</em></code>.</li>
<li><code>[[[[5,0],[7,4]],[5,5]],[6,6]]</code> becomes <code><em>1137</em></code>.</li>
<li><code>[[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]</code> becomes <code><em>3488</em></code>.</li>
</ul>
<p>So, given this example homework assignment:</p>
<pre><code>[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
</code></pre>
<p>The final sum is:</p>
<pre><code>[[[[6,6],[7,6]],[[7,7],[7,0]]],[[[7,7],[7,7]],[[7,8],[9,9]]]]</code></pre>
<p>The magnitude of this final sum is <code><em>4140</em></code>.</p>
<p>Add up all of the snailfish numbers from the homework assignment in the order they appear. <em>What is the magnitude of the final sum?</em></p>
</article>


In [11]:
from collections.abc import Iterable, Iterator
from enum import Enum
import json
from math import ceil

from PrettyPrint import PrettyPrintTree
from more_itertools import first
from numpy import isin
from printree import ptree

DO_PRINT = False


class ActionType(Enum):
    EXPLODE = 1
    LEFT_SPLIT = 2
    RIGHT_SPLIT = 3

    def __str__(self) -> str:
        if self.value == 1:
            return "explode"
        return "split"


@dataclass
class SnailFishNumber:
    left: SnailFishNumber | int
    right: SnailFishNumber | int
    parent: SnailFishNumber | None = None
    depth: int = 1

    def magnitude(self) -> int:
        left = self.left if isinstance(self.left, int) else self.left.magnitude()
        right = self.right if isinstance(self.right, int) else self.right.magnitude()
        return 3 * left + 2 * right

    def reduce(self) -> SnailFishNumber:
        if DO_PRINT:
            print(f"after addition:\t\t{self.to_list()}")
            self.print()
            print("\n\n")
        while (action := self.most_left_action()) is not None:
            type, snf = action
            match (type):
                case ActionType.EXPLODE:
                    snf.explode()
                case ActionType.LEFT_SPLIT:
                    snf.split(type)
                case ActionType.RIGHT_SPLIT:
                    snf.split(type)

            if DO_PRINT:
                print(f"after {type}:\t\t{self.to_list()}")
                self.print()
                print("\n\n")

        return self

    def most_left_action(self) -> tuple[ActionType, SnailFishNumber] | None:
        action = self.most_left_explosion()
        if action is not None:
            return action
        return self.most_left_split()

    def most_left_explosion(self) -> tuple[ActionType, SnailFishNumber] | None:
        def dfs(sfn) -> tuple[ActionType, SnailFishNumber] | None:

            if isinstance(sfn.left, SnailFishNumber):
                res = dfs(sfn.left)
                if res is not None:
                    return res

            if (
                sfn.depth >= 4
                and isinstance(sfn.left, int)
                and isinstance(sfn.right, int)
            ):
                return ActionType.EXPLODE, sfn

            if isinstance(sfn.right, SnailFishNumber):
                res = dfs(sfn.right)
                if res is not None:
                    return res

            return None

        return dfs(self)

    def most_left_split(self) -> tuple[ActionType, SnailFishNumber] | None:
        def dfs(sfn) -> tuple[ActionType, SnailFishNumber] | None:
            if isinstance(sfn.left, int) and sfn.left >= 10:
                return ActionType.LEFT_SPLIT, sfn

            if isinstance(sfn.left, SnailFishNumber):
                res = dfs(sfn.left)
                if res is not None:
                    return res

            if isinstance(sfn.right, int) and sfn.right >= 10:
                return ActionType.RIGHT_SPLIT, sfn

            if isinstance(sfn.right, SnailFishNumber):
                res = dfs(sfn.right)
                if res is not None:
                    return res

            return None

        return dfs(self)

    def split(self, action: ActionType):
        if action == ActionType.LEFT_SPLIT:
            self.left = SnailFishNumber(
                self.left // 2,
                int(ceil(self.left / 2)),
                self,
                self.depth + 1,
            )
        else:
            self.right = SnailFishNumber(
                self.right // 2,
                int(ceil(self.right / 2)),
                self,
                self.depth + 1,
            )

    def update_number_left(self, n: int) -> None:
        child = self
        parent = self.parent

        while parent is not None and parent.left is child:
            child = parent
            parent = child.parent

        if parent is None:
            return

        if isinstance(parent.left, int):
            parent.left += n
            return

        current = parent.left
        while True:
            if isinstance(current.right, int):
                current.right += n
                return
            current = current.right

    def update_number_right(self, n: int) -> None:
        child = self
        parent = self.parent

        while parent is not None and parent.right is child:
            child = parent
            parent = child.parent

        if parent is None:
            return

        if isinstance(parent.right, int):
            parent.right += n
            return

        current = parent.right
        while True:
            if isinstance(current.left, int):
                current.left += n
                return
            current = current.left

    def explode(self):
        self.update_number_left(self.left)
        self.update_number_right(self.right)
        if self.parent.left is self:
            self.parent.left = 0
        else:
            self.parent.right = 0

    def __add__(self, other: SnailFishNumber) -> SnailFishNumber:
        snf = SnailFishNumber(self, other, None, 0)
        self.parent = snf
        other.parent = snf
        self.update_depths()
        other.update_depths()
        return snf.reduce()

    def update_depths(self):
        self.depth += 1
        if isinstance(self.left, SnailFishNumber):
            self.left.update_depths()
        if isinstance(self.right, SnailFishNumber):
            self.right.update_depths()

    def print(self):
        class Tree:
            def __init__(self, value):
                self.val = value
                self.children = []

            def add_child(self, child):
                self.children.append(child)
                return child

        pt = PrettyPrintTree(lambda x: x.children, lambda x: x.val)
        root = Tree(f"d={self.depth}")

        stack = [(root, self.left, self.right)]
        while stack:
            parent, left, right = stack.pop()

            if isinstance(left, int):
                parent.add_child(Tree(f"{left}"))
            else:
                tree = Tree(f"d={left.depth}")
                parent.add_child(tree)
                stack.append((tree, left.left, left.right))

            if isinstance(right, int):
                parent.add_child(Tree(f"{right}"))
            else:
                tree = Tree(f"d={right.depth}")
                parent.add_child(tree)
                stack.append((tree, right.left, right.right))

        pt(root)

    def to_list(self) -> list:
        return [
            self.left if isinstance(self.left, int) else self.left.to_list(),
            self.right if isinstance(self.right, int) else self.right.to_list(),
        ]

    @classmethod
    def create(cls, data: str | list) -> SnailFishNumber:
        def dfs(data: list | int, depth: int) -> SnailFishNumber | int:
            if isinstance(data, int):
                return data

            left, right = data
            left = dfs(left, depth + 1)
            right = dfs(right, depth + 1)

            current = cls(left, right, depth=depth)

            if isinstance(left, cls):
                left.parent = current
            if isinstance(right, cls):
                right.parent = current
            return current

        if isinstance(data, str):
            data = json.loads(data)
        return dfs(data, 0)

    @classmethod
    def sum_from_string(cls, s: str) -> SnailFishNumber:
        snf_iter = (SnailFishNumber.create(l.strip()) for l in s.strip().splitlines())

        som = next(snf_iter)
        for snf in snf_iter:
            som = som + snf
        return som

    @classmethod
    def list_from_string(cls, s: str) -> Iterator[SnailFishNumber]:
        yield from (SnailFishNumber.create(l.strip()) for l in s.strip().splitlines())

In [12]:
tests = [
    {
        "name": "Example 1",
        "s": """
           [1,2]
           [[3,4],5]
        """,
        "expected": [[1, 2], [[3, 4], 5]],
    },
    {
        "name": "Example 2",
        "s": """
           [[[[4,3],4],4],[7,[[8,4],9]]]
           [1,1]
        """,
        "expected": [[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]],
    },
    {
        "name": "Example 3",
        "s": """
            [1,1]
            [2,2]
            [3,3]
            [4,4]
        """,
        "expected": [[[[1, 1], [2, 2]], [3, 3]], [4, 4]],
    },
    {
        "name": "Example 4",
        "s": """
            [1,1]
            [2,2]
            [3,3]
            [4,4]
            [5,5]
        """,
        "expected": [[[[3, 0], [5, 3]], [4, 4]], [5, 5]],
    },
    {
        "name": "Example 5",
        "s": """
            [1,1]
            [2,2]
            [3,3]
            [4,4]
            [5,5]
            [6,6]
        """,
        "expected": [[[[5, 0], [7, 4]], [5, 5]], [6, 6]],
    },
    {
        "name": "Example 6",
        "s": """
            [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
            [7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
            [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
            [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
            [7,[5,[[3,8],[1,4]]]]
            [[2,[2,2]],[8,[8,1]]]
            [2,9]
            [1,[[[9,3],9],[[9,0],[0,7]]]]
            [[[5,[7,4]],7],1]
            [[[[4,2],2],6],[8,7]]
        """,
        "expected": [[[[8, 7], [7, 7]], [[8, 6], [7, 7]]], [[[0, 7], [6, 6]], [8, 7]]],
    },
    {
        "name": "Example 6-1",
        "s": """
            [[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
            [7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
        """,
        "expected": [
            [[[4, 0], [5, 4]], [[7, 7], [6, 0]]],
            [[8, [7, 7]], [[7, 9], [5, 0]]],
        ],
    },
    {
        "name": "Example 6-2",
        "s": """
            [[[[4,0],[5,4]],[[7,7],[6,0]]],[[8,[7,7]],[[7,9],[5,0]]]]
            [[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
        """,
        "expected": [
            [[[6, 7], [6, 7]], [[7, 7], [0, 7]]],
            [[[8, 7], [7, 7]], [[8, 8], [8, 0]]],
        ],
    },
    {
        "name": "Example 6-3",
        "s": """
            [[[[6,7],[6,7]],[[7,7],[0,7]]],[[[8,7],[7,7]],[[8,8],[8,0]]]]
            [[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
        """,
        "expected": [
            [[[7, 0], [7, 7]], [[7, 7], [7, 8]]],
            [[[7, 7], [8, 8]], [[7, 7], [8, 7]]],
        ],
    },
    {
        "name": "Example 6-4",
        "s": """
            [[[[7,0],[7,7]],[[7,7],[7,8]]],[[[7,7],[8,8]],[[7,7],[8,7]]]]
            [7,[5,[[3,8],[1,4]]]]
        """,
        "expected": [
            [[[7, 7], [7, 8]], [[9, 5], [8, 7]]],
            [[[6, 8], [0, 8]], [[9, 9], [9, 0]]],
        ],
    },
    {
        "name": "Example 6-5",
        "s": """
            [[[[7,7],[7,8]],[[9,5],[8,7]]],[[[6,8],[0,8]],[[9,9],[9,0]]]]
            [[2,[2,2]],[8,[8,1]]]
        """,
        "expected": [
            [[[6, 6], [6, 6]], [[6, 0], [6, 7]]],
            [[[7, 7], [8, 9]], [8, [8, 1]]],
        ],
    },
    {
        "name": "Example 6-6",
        "s": """
            [[[[6,6],[6,6]],[[6,0],[6,7]]],[[[7,7],[8,9]],[8,[8,1]]]]
            [2,9]
        """,
        "expected": [[[[6, 6], [7, 7]], [[0, 7], [7, 7]]], [[[5, 5], [5, 6]], 9]],
    },
    {
        "name": "Example 6-7",
        "s": """
            [[[[6,6],[7,7]],[[0,7],[7,7]]],[[[5,5],[5,6]],9]]
            [1,[[[9,3],9],[[9,0],[0,7]]]]
        """,
        "expected": [
            [[[7, 8], [6, 7]], [[6, 8], [0, 8]]],
            [[[7, 7], [5, 0]], [[5, 5], [5, 6]]],
        ],
    },
    {
        "name": "Example 6-8",
        "s": """
            [[[[7,8],[6,7]],[[6,8],[0,8]]],[[[7,7],[5,0]],[[5,5],[5,6]]]]
            [[[5,[7,4]],7],1]
        """,
        "expected": [[[[7, 7], [7, 7]], [[8, 7], [8, 7]]], [[[7, 0], [7, 7]], 9]],
    },
    {
        "name": "Example 6-9",
        "s": """
            [[[[7,7],[7,7]],[[8,7],[8,7]]],[[[7,0],[7,7]],9]]
            [[[[4,2],2],6],[8,7]]
        """,
        "expected": [[[[8, 7], [7, 7]], [[8, 6], [7, 7]]], [[[0, 7], [6, 6]], [8, 7]]],
    },
    {
        "name": "Example 7",
        "s": """
            [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
            [[[5,[2,8]],4],[5,[[9,9],0]]]
            [6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
            [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
            [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
            [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
            [[[[5,4],[7,7]],8],[[8,3],8]]
            [[9,3],[[9,9],[6,[4,9]]]]
            [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
            [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
        """,
        "expected": [
            [[[6, 6], [7, 6]], [[7, 7], [7, 0]]],
            [[[7, 7], [7, 7]], [[7, 8], [9, 9]]],
        ],
    },
]


def snailfishnumber_addition_test(s: str) -> list:
    som = SnailFishNumber.sum_from_string(s)
    return som.to_list()


run_tests_params(snailfishnumber_addition_test, tests)


Test Example 1 passed, for snailfishnumber_addition_test.
Test Example 2 passed, for snailfishnumber_addition_test.
Test Example 3 passed, for snailfishnumber_addition_test.
Test Example 4 passed, for snailfishnumber_addition_test.
Test Example 5 passed, for snailfishnumber_addition_test.
Test Example 6 passed, for snailfishnumber_addition_test.
Test Example 6-1 passed, for snailfishnumber_addition_test.
Test Example 6-2 passed, for snailfishnumber_addition_test.
Test Example 6-3 passed, for snailfishnumber_addition_test.
Test Example 6-4 passed, for snailfishnumber_addition_test.
Test Example 6-5 passed, for snailfishnumber_addition_test.
Test Example 6-6 passed, for snailfishnumber_addition_test.
Test Example 6-7 passed, for snailfishnumber_addition_test.
Test Example 6-8 passed, for snailfishnumber_addition_test.
Test Example 6-9 passed, for snailfishnumber_addition_test.
Test Example 7 passed, for snailfishnumber_addition_test.
Success


In [13]:
tests = [
    {
        "name": "Example 1",
        "snf": SnailFishNumber.create([9, 1]),
        "expected": 29,
    },
    {
        "name": "Example 2",
        "snf": SnailFishNumber.create([1, 9]),
        "expected": 21,
    },
    {
        "name": "Example 3",
        "snf": SnailFishNumber.create([[9, 1], [1, 9]]),
        "expected": 129,
    },
    {
        "name": "Example 4",
        "snf": SnailFishNumber.create([[1, 2], [[3, 4], 5]]),
        "expected": 143,
    },
    {
        "name": "Example 5",
        "snf": SnailFishNumber.create([[[[0, 7], 4], [[7, 8], [6, 0]]], [8, 1]]),
        "expected": 1384,
    },
    {
        "name": "Example 6",
        "snf": SnailFishNumber.create([[[[1, 1], [2, 2]], [3, 3]], [4, 4]]),
        "expected": 445,
    },
    {
        "name": "Example 7",
        "snf": SnailFishNumber.create([[[[3, 0], [5, 3]], [4, 4]], [5, 5]]),
        "expected": 791,
    },
    {
        "name": "Example 8",
        "snf": SnailFishNumber.create([[[[5, 0], [7, 4]], [5, 5]], [6, 6]]),
        "expected": 1137,
    },
    {
        "name": "Example 9",
        "snf": SnailFishNumber.create(
            [[[[8, 7], [7, 7]], [[8, 6], [7, 7]]], [[[0, 7], [6, 6]], [8, 7]]]
        ),
        "expected": 3488,
    },
    {
        "name": "Example 4",
        "snf": SnailFishNumber.create(
            [[[[6, 6], [7, 6]], [[7, 7], [7, 0]]], [[[7, 7], [7, 7]], [[7, 8], [9, 9]]]]
        ),
        "expected": 4140,
    },
]


def snailfishnumber_magnitude_test(snf: SnailFishNumber) -> int:
    return snf.magnitude()


run_tests_params(snailfishnumber_magnitude_test, tests)

homework_example = """
    [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
    [[[5,[2,8]],4],[5,[[9,9],0]]]
    [6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
    [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
    [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
    [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
    [[[[5,4],[7,7]],8],[[8,3],8]]
    [[9,3],[[9,9],[6,[4,9]]]]
    [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
    [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
"""

assert SnailFishNumber.sum_from_string(homework_example).magnitude() == 4140


Test Example 1 passed, for snailfishnumber_magnitude_test.
Test Example 2 passed, for snailfishnumber_magnitude_test.
Test Example 3 passed, for snailfishnumber_magnitude_test.
Test Example 4 passed, for snailfishnumber_magnitude_test.
Test Example 5 passed, for snailfishnumber_magnitude_test.
Test Example 6 passed, for snailfishnumber_magnitude_test.
Test Example 7 passed, for snailfishnumber_magnitude_test.
Test Example 8 passed, for snailfishnumber_magnitude_test.
Test Example 9 passed, for snailfishnumber_magnitude_test.
Test Example 4 passed, for snailfishnumber_magnitude_test.
Success


In [14]:
with open("../input/day18.txt") as f:
    puzzle = f.read()

print(f"Part I: {SnailFishNumber.sum_from_string(puzzle).magnitude()}")

Part I: 4641


<link href="style.css" rel="stylesheet"></link>

<main>

<p>Your puzzle answer was <code>4641</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You notice a second question on the back of the homework assignment:</p>
<p>What is the largest magnitude you can get from adding only two of the snailfish numbers?</p>
<p>Note that snailfish addition is not <a href="https://en.wikipedia.org/wiki/Commutative_property" target="_blank">commutative</a> - that is, <code>x + y</code> and <code>y + x</code> can produce different results.</p>
<p>Again considering the last example homework assignment above:</p>
<pre><code>[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
</code></pre>
<p>The largest magnitude of the sum of any two snailfish numbers in this list is <code><em>3993</em></code>. This is the magnitude of <code>[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]</code> + <code>[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]</code>, which reduces to <code>[[[[7,8],[6,6]],[[6,0],[7,7]]],[[[7,8],[8,8]],[[7,9],[0,6]]]]</code>.</p>
<p><em>What is the largest magnitude of any sum of two different snailfish numbers from the homework assignment?</em></p>
</article>

</main>


In [15]:
from copy import deepcopy


example = """
    [[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
    [[[5,[2,8]],4],[5,[[9,9],0]]]
    [6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
    [[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
    [[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
    [[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
    [[[[5,4],[7,7]],8],[[8,3],8]]
    [[9,3],[[9,9],[6,[4,9]]]]
    [[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
    [[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]
"""


def max_magnitude_sum_of_two_from_list(s):
    snfs = list(SnailFishNumber.list_from_string(s))
    return max(
        (deepcopy(snf1) + deepcopy(snf2)).magnitude()
        for snf1, snf2 in product(snfs, snfs)
        if snf1 is not snf2
    )


assert max_magnitude_sum_of_two_from_list(example) == 3993
#

In [16]:
print(f"Part II: { max_magnitude_sum_of_two_from_list(puzzle) }")

Part II: 4624


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>4624</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
